In [1]:
import math
import geocoder
import folium
from geopy.distance import distance
from math import radians, sin, cos, sqrt, atan, degrees

# Calculating the FOV of the Main Camera on Samsung Note 20 Ultra and Boundary Angles


## 1. Technical Data of the Main Camera
- **Focal length:** 26 mm (35mm equivalent).
- **Sensor:** 1/1.33 inch (diagonal ~19.1 mm).
- **Sensor ratio:** 1:1.
  - **Sensor width:** $ 19.1 \cdot \frac{1}{\sqrt{2}} \text{mm} $.
  - **Sensor height:** $ 19.1 \cdot \frac{1}{\sqrt{2}} \text{mm} $.

## 2. FOV Calculation Formula
$$
\text{FOV} = 2 \cdot \arctan\left(\frac{\text{sensor size}}{2 \cdot \text{focal length}}\right)
$$

### Horizontal FOV (HFOV):
$$
\text{HFOV} = 2 \cdot \arctan\left(\frac{\text{Sensor width}}{2 \cdot \text{focal length}}\right)
$$

### Vertical FOV (VFOV):
$$
\text{VFOV} = 2 \cdot \arctan\left(\frac{\text{sensor height}}{2 \cdot \text{focal length}}\right)
$$

### Diagonal FOV (DFOV):
$$
\text{DFOV} = 2 \cdot \arctan\left(\frac{\text{sensor diagonal}}{2 \cdot \text{focal length}}\right)
$$


## 3. Calculating Left and Right Boundary Angles (Horizontal)
- **Central orientation (θc):** mesured by Compass.
- **Left boundary angle (θ_left):**
$$
\theta_{\text{left}} = \theta_c - \frac{\text{HFOV}}{2}
$$
- **Right boundary angle (θ_right):**
$$
\theta_{\text{right}} = \theta_c + \frac{\text{HFOV}}{2}
$$

In [2]:
def calculate_fov(sensor_size, focal_length):
    """
    Calculate the Field of View (FOV).

    :param sensor_size: Size of the sensor (in mm).
    :param focal_length: Focal length of the lens (in mm).
    :return: FOV in degrees.
    """
    fov = 2 * math.degrees(atan(sensor_size / (2 * focal_length)))
    return fov

def calculate_angles(center_angle, fov):
    """
    Calculate the left and right boundary angles based on the center angle and FOV.

    :param center_angle: Center orientation angle (in degrees).
    :param fov: Field of View (in degrees).
    :return: Left and right angles.
    """
    left_angle = center_angle - (fov / 2)
    right_angle = center_angle + (fov / 2)
    return left_angle, right_angle

In [3]:
# Sensor dimensions and focal length
sensor_diagonal = 19.1  # in mm
sensor_ratio = (1, 1)   # Aspect ratio of the sensor
sensor_width = sensor_diagonal * (sensor_ratio[0] / sqrt(sensor_ratio[0]**2 + sensor_ratio[1]**2))
sensor_height = sensor_diagonal * (sensor_ratio[1] / sqrt(sensor_ratio[0]**2 + sensor_ratio[1]**2))
focal_length = 26       # in mm

# Calculate FOVs
hfov = calculate_fov(sensor_width, focal_length)   # Horizontal FOV
vfov = calculate_fov(sensor_height, focal_length)  # Vertical FOV
diagonal_fov = calculate_fov(sensor_diagonal, focal_length)  # Diagonal FOV


# Print results
print("Horizontal FOV (HFOV):", round(hfov, 2), "degrees")
print("Vertical FOV (VFOV):", round(vfov, 2), "degrees")
print("Diagonal FOV:", round(diagonal_fov, 2), "degrees")

Horizontal FOV (HFOV): 29.12 degrees
Vertical FOV (VFOV): 29.12 degrees
Diagonal FOV: 40.34 degrees


### **Quy trình sử dụng Compass và chụp ảnh bầu trời**
1. **Xác định hướng (trước khi dựng điện thoại lên)**:
   - Đặt điện thoại **song song với mặt đất**.
   - Dùng Compass để ghi lại hướng trung tâm (θc), ví dụ: 45° (Đông Bắc).
   - Điều này giúp đảm bảo thông tin đầu vào chính xác cho việc tính toán góc biên trái và phải.

2. **Dựng điện thoại để chụp ảnh**:
   - Sau khi biết hướng trung tâm, bạn có thể dựng điện thoại theo góc phù hợp để chụp bầu trời.
   - Cách đặt:
     - **Thẳng đứng (90°)** nếu muốn tập trung hoàn toàn vào bầu trời.
     - **Nghiêng nhẹ** nếu muốn bao quát cả đường chân trời.

3. **Tính toán vùng quan sát**:
   - Dựa trên hướng trung tâm đo được và góc mở của camera (FOV), tính toán góc biên để xác định vùng bầu trời trong ảnh.

---

### **Lý do không dùng Compass khi điện thoại dựng đứng**
- Khi điện thoại dựng đứng, cảm biến từ trường đo hướng **theo phương dọc** không đáng tin cậy do nhiễu từ trường theo phương khác.
- Kết quả có thể sai lệch, ảnh hưởng đến độ chính xác khi tính toán góc biên.

---

**Tóm lại**: Đặt điện thoại **song song với mặt đất để đo hướng**, sau đó dựng lên để chụp ảnh. Điều này giúp đảm bảo cả dữ liệu đầu vào và bức ảnh đều chính xác.

In [8]:
# Center orientation angle (e.g., facing 45 degrees)
center_orientation = 310  # in degrees
print("Center orientation:", center_orientation, "degrees")

# Calculate left and right boundary angles
left_angle, right_angle = calculate_angles(center_orientation, hfov)

print("Left boundary angle:", round(left_angle, 2), "degrees")
print("Right boundary angle:", round(right_angle, 2), "degrees")

Center orientation: 310 degrees
Left boundary angle: 295.44 degrees
Right boundary angle: 324.56 degrees


#### **Thay đổi vĩ độ $( \Delta \text{lat} $):**
$$
\Delta \text{lat} = \frac{D}{R} \cdot \cos(\theta)
$$

#### **Thay đổi kinh độ ($ \Delta \text{lon} $):**
$$
\Delta \text{lon} = \frac{D}{R \cdot \cos(\text{latitude})} \cdot \sin(\theta)
$$

- $ D $: Khoảng cách quan sát tối đa (tính bằng mét).
- $ R $: Bán kính Trái Đất (6,371 km = 6,371,000 m).
- $ \theta $: Góc (hướng biên trái hoặc phải, tính bằng radian).
- $ \text{latitude} $: Vĩ độ hiện tại của camera.

In [9]:
def calculate_gps_coordinates(lat0, lon0, distance, angle, earth_radius=6_371_000):
    """
    Calculate GPS coordinates based on distance and angle.
    
    :param lat0: Initial latitude in degrees.
    :param lon0: Initial longitude in degrees.
    :param distance: Distance in meters.
    :param angle: Angle in degrees.
    :param earth_radius: Radius of the Earth in meters (default is 6,371,000 m).
    :return: New latitude and longitude in degrees.
    """
    angle_rad = radians(angle)
    lat0_rad = radians(lat0)
    lon0_rad = radians(lon0)
    
    delta_lat = (distance / earth_radius) * cos(angle_rad)
    new_lat = lat0_rad + delta_lat
    
    delta_lon = (distance / (earth_radius * cos(lat0_rad))) * sin(angle_rad)
    new_lon = lon0_rad + delta_lon
    
    return degrees(new_lat), degrees(new_lon)

In [10]:
# Function to calculate GPS coordinates based on distance and angle
def calculate_coordinates(lat, lon, dist, angle):
    """
    Calculate new GPS coordinates based on distance and angle.
    :param lat: Latitude of the starting point.
    :param lon: Longitude of the starting point.
    :param dist: Distance in meters.
    :param angle: Angle in degrees (bearing).
    :return: Tuple of new latitude and longitude.
    """
    point = distance(meters=dist).destination((lat, lon), bearing=angle)
    return point.latitude, point.longitude

Công thức $ D = \sqrt{2 \cdot R \cdot h} $ xuất phát từ hình học cầu, được dùng để tính khoảng cách từ một vị trí quan sát trên bề mặt Trái Đất đến đường chân trời.

1. **Mô hình hình học**:
   - Trái Đất được coi là một hình cầu với bán kính $ R $.
   - Một điểm quan sát ở độ cao $ h $ nằm ngoài bề mặt Trái Đất.
   - Đường chân trời được xác định là giao điểm của đường thẳng đi qua mắt quan sát và bề mặt Trái Đất.

2. **Phương trình hình học**:
   - Từ định lý Pythagoras, với tam giác vuông tạo bởi:
     - $ R $: Bán kính Trái Đất.
     - $ R + h $: Khoảng cách từ tâm Trái Đất đến điểm quan sát.
     - $ D $: Khoảng cách từ điểm quan sát đến đường chân trời.
   - Quan hệ: $(R + h)^2 = R^2 + D^2$.

3. **Đơn giản hóa**:
   - Vì $ h \ll R $ (độ cao nhỏ hơn rất nhiều so với bán kính Trái Đất), có thể xấp xỉ:
     $$
     R + h \approx R
     $$
   - Suy ra:
     $$
     D^2 \approx 2 \cdot R \cdot h
     $$
   - Do đó:
     $$
     D = \sqrt{2 \cdot R \cdot h}
     $$

In [17]:
# Get current GPS coordinates
#g = geocoder.ip('me')  # Get location based on IP address
#lat1, lon1 = g.latlng
lat1, lon1 = 43.57072605547293, 1.4682620342267738

# Increase distance for observation from a high floor
observation_distance = 20_000  # meters

# Calculate GPS coordinates for left and right boundaries
left_coordinates = calculate_gps_coordinates(lat1, lon1, observation_distance, left_angle)
right_coordinates = calculate_gps_coordinates(lat1, lon1, observation_distance, right_angle)

# Print results
print("Current GPS coordinates:", (lat1, lon1))
print("Left boundary GPS coordinates:", left_coordinates)
print("Right boundary GPS coordinates:", right_coordinates)

Current GPS coordinates: (43.57072605547293, 1.4682620342267738)
Left boundary GPS coordinates: (43.6479910190868, 1.244082868618402)
Right boundary GPS coordinates: (43.717264772687855, 1.3243115044058589)


In [12]:
# Calculate boundary coordinates
boundary_points = []
for angle in range(int(left_angle), int(right_angle) + 1):
    boundary_points.append(calculate_gps_coordinates(lat1, lon1, observation_distance, angle))

# Add starting and ending points to close the polygon
boundary_points.insert(0, (lat1, lon1))
boundary_points.append((lat1, lon1))

# Create a folium map
m = folium.Map(location=[lat1, lon1], zoom_start=14)

# Add the contour area to the map
folium.Polygon(
    locations=boundary_points,
    color="blue",
    weight=2,
    fill=True,
    fill_color="blue",
    fill_opacity=0.4,
).add_to(m)

# Add a marker for the starting point
folium.Marker(
    [lat1, lon1],
    popup="Observation Point",
    icon=folium.Icon(color="red", icon="info-sign")
).add_to(m)

m

In [36]:
# Initial camera 1 coordinates and parameters
h = 10000  # Altitude in meters
FOV = 60  # Field of View in degrees
FOV_overlap = 20  # Overlap Field of View in degrees
center_orientation1 = 310  # Orientation of Camera 1 in degrees

# Calculate optimal distance
d = h * math.tan(math.radians(FOV / 2)) + h * math.tan(math.radians(FOV_overlap / 2))
d = 2 * h * math.tan(math.radians(FOV / 2))

# Calculate new camera 2 position
lat2, lon2 = calculate_gps_coordinates(lat1, lon1, d, center_orientation1)

# Calculate new camera 2 orientation
center_orientation2 = (center_orientation1 + 180) % 360

# Print results
print("Camera 2 Coordinates:", (lat2, lon2))
print("Camera 2 Orientation:", round(center_orientation2, 2), "degrees")
print(f"Optimal distance between cameras: {d / 1000:.2f} km")

Camera 2 Coordinates: (43.63747615112623, 1.3584662869373343)
Camera 2 Orientation: 130 degrees
Optimal distance between cameras: 11.55 km


In [37]:
# Calculate Camera 2 boundary
left_angle2 = center_orientation2 - (hfov / 2)
right_angle2 = center_orientation2 + (hfov / 2)
boundary_points2 = [calculate_gps_coordinates(lat2, lon2, observation_distance, angle) for angle in range(int(left_angle2), int(right_angle2) + 1)]
boundary_points2.insert(0, (lat2, lon2))
boundary_points2.append((lat2, lon2))

# Create folium map
m = folium.Map(location=[lat1, lon1], zoom_start=12)

# Add Camera 1 observation area
folium.Polygon(
    locations=boundary_points,
    color="blue",
    weight=2,
    fill=True,
    fill_color="blue",fill_opacity=0.4,
    popup="Camera 1 Area"
).add_to(m)

# Add Camera 2 observation area
folium.Polygon(
    locations=boundary_points2,
    color="green",
    weight=2,
    fill=True,
    fill_color="green",
    fill_opacity=0.4,
    popup="Camera 2 Area"
).add_to(m)

# Add markers for camera locations
folium.Marker([lat1, lon1], popup="Camera 1", icon=folium.Icon(color="red")).add_to(m)
folium.Marker([lat2, lon2], popup="Camera 2", icon=folium.Icon(color="blue")).add_to(m)

# Save and display map
m